# Chapter 4: Memory Management 실습

이 노트북은 LangChain과 LangGraph에서 메모리를 관리하는 다양한 방법을 실습합니다.

## 환경 설정

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = input("OpenAI API Key를 입력하세요: ")

## 1. 간단한 메모리 구현

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from typing import List

# 간단한 메모리 클래스
class SimpleMemory:
    def __init__(self, max_messages: int = 10):
        self.messages: List = []
        self.max_messages = max_messages
    
    def add_message(self, message):
        self.messages.append(message)
        # 최대 메시지 수 유지
        if len(self.messages) > self.max_messages:
            self.messages = self.messages[-self.max_messages:]
    
    def get_messages(self):
        return self.messages
    
    def clear(self):
        self.messages = []

# 메모리가 있는 챗봇
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
memory = SimpleMemory()

def chat_with_memory(user_input: str):
    # 시스템 메시지 추가
    system_msg = SystemMessage(content="당신은 친절한 AI 어시스턴트입니다. 대화 내용을 기억하고 있습니다.")
    
    # 사용자 메시지 추가
    memory.add_message(HumanMessage(content=user_input))
    
    # 전체 대화 기록으로 응답 생성
    messages = [system_msg] + memory.get_messages()
    response = llm.invoke(messages)
    
    # AI 응답 메모리에 저장
    memory.add_message(response)
    
    return response.content

# 대화 테스트
print("챗봇: 안녕하세요! 무엇을 도와드릴까요?\n")

# 대화 시뮬레이션
conversations = [
    "제 이름은 철수입니다.",
    "저는 프로그래밍을 배우고 있어요.",
    "제 이름이 뭐라고 했죠?"
]

for user_input in conversations:
    print(f"사용자: {user_input}")
    response = chat_with_memory(user_input)
    print(f"챗봇: {response}\n")

챗봇: 안녕하세요! 무엇을 도와드릴까요?

사용자: 제 이름은 철수입니다.
챗봇: 안녕하세요, 철수님! 만나서 반갑습니다. 어떻게 도와드릴까요?

사용자: 저는 프로그래밍을 배우고 있어요.
챗봇: 멋지네요, 철수님! 어떤 프로그래밍 언어를 배우고 계신가요? 또는 특정한 프로젝트나 목표가 있으신가요?

사용자: 제 이름이 뭐라고 했죠?
챗봇: 당신의 이름은 철수입니다! 어떻게 도와드릴까요?



## 2. LangGraph State 기반 메모리

In [3]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, END
import operator

# State 정의
class ConversationState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    user_name: str
    conversation_count: int

# 노드 함수들
def process_input(state: ConversationState):
    """사용자 입력 처리"""
    # 대화 횟수 증가
    state["conversation_count"] = state.get("conversation_count", 0) + 1
    
    # 사용자 이름 추출 (간단한 예제)
    last_message = state["messages"][-1].content if state["messages"] else ""
    if "제 이름은" in last_message:
        name = last_message.split("제 이름은")[1].split("입니다")[0].strip()
        state["user_name"] = name
    
    return state

def generate_response(state: ConversationState):
    """응답 생성"""
    llm = ChatOpenAI(model="gpt-4o-mini")
    
    # 시스템 메시지 구성
    system_content = "당신은 친절한 AI 어시스턴트입니다."
    if state.get("user_name"):
        system_content += f" 사용자의 이름은 {state['user_name']}입니다."
    system_content += f" 이것은 {state.get('conversation_count', 1)}번째 대화입니다."
    
    messages = [SystemMessage(content=system_content)] + list(state["messages"])
    response = llm.invoke(messages)
    
    return {"messages": [response]}

# 그래프 구성
workflow = StateGraph(ConversationState)

# 노드 추가
workflow.add_node("process", process_input)
workflow.add_node("respond", generate_response)

# 엣지 추가
workflow.set_entry_point("process")
workflow.add_edge("process", "respond")
workflow.add_edge("respond", END)

# 그래프 컴파일
app = workflow.compile()

# 대화 실행
state = {"messages": [], "user_name": "", "conversation_count": 0}

test_messages = [
    "안녕하세요!",
    "제 이름은 영희입니다.",
    "오늘 날씨가 좋네요.",
    "제 이름을 기억하시나요?"
]

for msg in test_messages:
    print(f"사용자: {msg}")
    state["messages"].append(HumanMessage(content=msg))
    result = app.invoke(state)
    state = result  # 상태 업데이트
    print(f"AI: {result['messages'][-1].content}")
    print(f"(저장된 이름: {result.get('user_name', '없음')}, 대화 횟수: {result['conversation_count']})\n")

사용자: 안녕하세요!
AI: 안녕하세요! 어떻게 도와드릴까요?
(저장된 이름: , 대화 횟수: 1)

사용자: 제 이름은 영희입니다.
AI: 영희님, 반갑습니다! 무엇을 도와드릴까요?
(저장된 이름: 영희, 대화 횟수: 2)

사용자: 오늘 날씨가 좋네요.
AI: 맞아요! 날씨가 좋으면 기분도 좋고 외출하기에도 좋죠. 영희님은 오늘 어떤 계획 있으신가요?
(저장된 이름: 영희, 대화 횟수: 3)

사용자: 제 이름을 기억하시나요?
AI: 네, 영희님! 기억하고 있습니다. 계속해서 무엇을 도와드릴까요?
(저장된 이름: 영희, 대화 횟수: 4)



## 3. 영구 메모리 (Checkpointing)

In [4]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, MessagesState
import uuid

# 메모리 세이버 생성
memory_saver = MemorySaver()

# 간단한 챗봇 노드
def chatbot(state: MessagesState):
    llm = ChatOpenAI(model="gpt-4o-mini")
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

# 그래프 구성
workflow = StateGraph(MessagesState)
workflow.add_node("chatbot", chatbot)
workflow.set_entry_point("chatbot")
workflow.add_edge("chatbot", END)

# 체크포인터와 함께 컴파일
app_with_memory = workflow.compile(checkpointer=memory_saver)

# 세션 ID 생성
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

print(f"세션 ID: {thread_id}\n")

# 첫 번째 대화
messages1 = [HumanMessage(content="안녕하세요! 제가 좋아하는 색은 파란색입니다.")]
result1 = app_with_memory.invoke({"messages": messages1}, config)
print("첫 번째 대화:")
print(f"사용자: {messages1[0].content}")
print(f"AI: {result1['messages'][-1].content}\n")

# 두 번째 대화 (이전 대화 기억)
messages2 = [HumanMessage(content="제가 좋아하는 색이 뭐라고 했죠?")]
result2 = app_with_memory.invoke({"messages": messages2}, config)
print("두 번째 대화:")
print(f"사용자: {messages2[0].content}")
print(f"AI: {result2['messages'][-1].content}\n")

# 대화 기록 확인
state_snapshot = app_with_memory.get_state(config)
print("저장된 대화 기록:")
for msg in state_snapshot.values["messages"]:
    role = "사용자" if isinstance(msg, HumanMessage) else "AI"
    print(f"{role}: {msg.content}")

세션 ID: 9e777a2d-63b4-4b22-86b7-18aa3f6aa17a

첫 번째 대화:
사용자: 안녕하세요! 제가 좋아하는 색은 파란색입니다.
AI: 안녕하세요! 파란색은 정말 아름다운 색이죠. 상쾌하고 평온한 느낌을 주잖아요. 파란색을 좋아하시는 특별한 이유가 있으신가요?

두 번째 대화:
사용자: 제가 좋아하는 색이 뭐라고 했죠?
AI: 당신이 좋아하는 색은 파란색이라고 하셨습니다!

저장된 대화 기록:
사용자: 안녕하세요! 제가 좋아하는 색은 파란색입니다.
AI: 안녕하세요! 파란색은 정말 아름다운 색이죠. 상쾌하고 평온한 느낌을 주잖아요. 파란색을 좋아하시는 특별한 이유가 있으신가요?
사용자: 제가 좋아하는 색이 뭐라고 했죠?
AI: 당신이 좋아하는 색은 파란색이라고 하셨습니다!


## 4. 메시지 트리밍

In [5]:
from langchain_core.messages import trim_messages, SystemMessage

# 긴 대화 기록 생성
long_conversation = [
    SystemMessage(content="당신은 도움이 되는 AI입니다."),
    HumanMessage(content="안녕하세요!"),
    AIMessage(content="안녕하세요! 무엇을 도와드릴까요?"),
    HumanMessage(content="파이썬에 대해 알려주세요."),
    AIMessage(content="파이썬은 고급 프로그래밍 언어입니다..."),
    HumanMessage(content="파이썬의 장점은?"),
    AIMessage(content="파이썬의 장점은 읽기 쉽고..."),
    HumanMessage(content="파이썬으로 뭘 만들 수 있나요?"),
    AIMessage(content="웹 개발, 데이터 분석, AI 등..."),
    HumanMessage(content="더 자세히 설명해주세요."),
]

# 토큰 기반 트리밍
trimmed_by_tokens = trim_messages(
    long_conversation,
    max_tokens=100,
    strategy="last",
    token_counter=len,  # 실제로는 tiktoken 사용
    include_system=True
)

print("토큰 기반 트리밍 (최대 100토큰):")
for msg in trimmed_by_tokens:
    role = msg.__class__.__name__.replace("Message", "")
    print(f"{role}: {msg.content[:50]}...")

# 메시지 수 기반 트리밍
def trim_by_message_count(messages, max_messages=5, keep_system=True):
    """최근 N개 메시지만 유지"""
    system_msgs = [m for m in messages if isinstance(m, SystemMessage)]
    other_msgs = [m for m in messages if not isinstance(m, SystemMessage)]
    
    if keep_system and system_msgs:
        return system_msgs[:1] + other_msgs[-max_messages:]
    return messages[-max_messages:]

trimmed_by_count = trim_by_message_count(long_conversation, max_messages=4)

print("\n메시지 수 기반 트리밍 (최근 4개):")
for msg in trimmed_by_count:
    role = msg.__class__.__name__.replace("Message", "")
    print(f"{role}: {msg.content[:50]}...")

토큰 기반 트리밍 (최대 100토큰):
System: 당신은 도움이 되는 AI입니다....
Human: 안녕하세요!...
AI: 안녕하세요! 무엇을 도와드릴까요?...
Human: 파이썬에 대해 알려주세요....
AI: 파이썬은 고급 프로그래밍 언어입니다......
Human: 파이썬의 장점은?...
AI: 파이썬의 장점은 읽기 쉽고......
Human: 파이썬으로 뭘 만들 수 있나요?...
AI: 웹 개발, 데이터 분석, AI 등......
Human: 더 자세히 설명해주세요....

메시지 수 기반 트리밍 (최근 4개):
System: 당신은 도움이 되는 AI입니다....
AI: 파이썬의 장점은 읽기 쉽고......
Human: 파이썬으로 뭘 만들 수 있나요?...
AI: 웹 개발, 데이터 분석, AI 등......
Human: 더 자세히 설명해주세요....


## 5. 메시지 필터링

In [6]:
from langchain_core.messages import filter_messages
from typing import List

# 다양한 메시지 타입이 있는 대화
mixed_messages = [
    SystemMessage(content="시스템 설정", id="sys1"),
    HumanMessage(content="첫 번째 질문", id="human1"),
    AIMessage(content="첫 번째 답변", id="ai1"),
    HumanMessage(content="두 번째 질문", id="human2"),
    AIMessage(content="두 번째 답변", id="ai2"),
    SystemMessage(content="시스템 업데이트", id="sys2"),
    HumanMessage(content="세 번째 질문", id="human3"),
    AIMessage(content="세 번째 답변", id="ai3"),
]

# 특정 타입만 필터링
human_messages = filter_messages(mixed_messages, include_types=[HumanMessage])
print("사용자 메시지만:")
for msg in human_messages:
    print(f"- {msg.content}")

# 특정 ID 필터링
filtered_by_id = filter_messages(mixed_messages, include_ids=["human2", "ai2"])
print("\n특정 ID 메시지만:")
for msg in filtered_by_id:
    print(f"- [{msg.id}] {msg.content}")

# 커스텀 필터 함수
def custom_filter(messages: List[BaseMessage]) -> List[BaseMessage]:
    """질문과 답변 쌍만 유지"""
    result = []
    for i in range(len(messages)):
        if isinstance(messages[i], HumanMessage):
            result.append(messages[i])
            # 다음 메시지가 AI 응답이면 포함
            if i + 1 < len(messages) and isinstance(messages[i + 1], AIMessage):
                result.append(messages[i + 1])
    return result

qa_pairs = custom_filter(mixed_messages)
print("\nQ&A 쌍만:")
for i in range(0, len(qa_pairs), 2):
    if i + 1 < len(qa_pairs):
        print(f"Q: {qa_pairs[i].content}")
        print(f"A: {qa_pairs[i+1].content}")
        print()

사용자 메시지만:
- 첫 번째 질문
- 두 번째 질문
- 세 번째 질문

특정 ID 메시지만:
- [human2] 두 번째 질문
- [ai2] 두 번째 답변

Q&A 쌍만:
Q: 첫 번째 질문
A: 첫 번째 답변

Q: 두 번째 질문
A: 두 번째 답변

Q: 세 번째 질문
A: 세 번째 답변



## 6. 메시지 병합

In [7]:
from langchain_core.messages import merge_message_runs

# 연속된 같은 타입 메시지들
consecutive_messages = [
    HumanMessage(content="안녕하세요."),
    HumanMessage(content="질문이 있습니다."),
    HumanMessage(content="파이썬에 대해 알려주세요."),
    AIMessage(content="안녕하세요!"),
    AIMessage(content="파이썬은 프로그래밍 언어입니다."),
    AIMessage(content="1991년에 개발되었습니다."),
    HumanMessage(content="감사합니다."),
]

# 연속된 메시지 병합
merged = merge_message_runs(consecutive_messages)

print("병합 전:")
for msg in consecutive_messages:
    role = "사용자" if isinstance(msg, HumanMessage) else "AI"
    print(f"{role}: {msg.content}")

print("\n병합 후:")
for msg in merged:
    role = "사용자" if isinstance(msg, HumanMessage) else "AI"
    print(f"{role}: {msg.content}")

# 커스텀 병합 함수
def smart_merge(messages: List[BaseMessage]) -> List[BaseMessage]:
    """스마트 메시지 병합 (구분자 추가)"""
    if not messages:
        return []
    
    result = []
    current = messages[0]
    
    for msg in messages[1:]:
        if type(msg) == type(current):
            # 같은 타입이면 병합
            current.content += "\n" + msg.content
        else:
            # 다른 타입이면 현재 메시지 저장하고 새로 시작
            result.append(current)
            current = msg
    
    result.append(current)
    return result

smart_merged = smart_merge(consecutive_messages)
print("\n스마트 병합 (줄바꿈 포함):")
for msg in smart_merged:
    role = "사용자" if isinstance(msg, HumanMessage) else "AI"
    print(f"{role}:\n{msg.content}\n")

병합 전:
사용자: 안녕하세요.
사용자: 질문이 있습니다.
사용자: 파이썬에 대해 알려주세요.
AI: 안녕하세요!
AI: 파이썬은 프로그래밍 언어입니다.
AI: 1991년에 개발되었습니다.
사용자: 감사합니다.

병합 후:
사용자: 안녕하세요.
질문이 있습니다.
파이썬에 대해 알려주세요.
AI: 안녕하세요!
파이썬은 프로그래밍 언어입니다.
1991년에 개발되었습니다.
사용자: 감사합니다.

스마트 병합 (줄바꿈 포함):
사용자:
안녕하세요.
질문이 있습니다.
파이썬에 대해 알려주세요.

AI:
안녕하세요!
파이썬은 프로그래밍 언어입니다.
1991년에 개발되었습니다.

사용자:
감사합니다.



## 7. 요약 기반 메모리

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

class SummarizationMemory:
    def __init__(self, llm, max_messages=10):
        self.llm = llm
        self.messages = []
        self.summary = ""
        self.max_messages = max_messages
        
        # 요약 프롬프트
        self.summarize_prompt = ChatPromptTemplate.from_messages([
            ("system", "다음 대화를 간결하게 요약하세요. 중요한 정보와 맥락을 포함하세요."),
            ("human", "{conversation}")
        ])
        self.summarize_chain = self.summarize_prompt | llm | StrOutputParser()
    
    def add_exchange(self, human_msg: str, ai_msg: str):
        self.messages.extend([
            HumanMessage(content=human_msg),
            AIMessage(content=ai_msg)
        ])
        
        # 메시지가 많아지면 요약
        if len(self.messages) > self.max_messages:
            self._summarize_and_trim()
    
    def _summarize_and_trim(self):
        # 대화를 문자열로 변환
        conversation = "\n".join([
            f"{'사용자' if isinstance(m, HumanMessage) else 'AI'}: {m.content}"
            for m in self.messages[:-4]  # 최근 4개는 유지
        ])
        
        # 이전 요약이 있으면 포함
        if self.summary:
            conversation = f"이전 요약: {self.summary}\n\n새 대화:\n{conversation}"
        
        # 요약 생성
        self.summary = self.summarize_chain.invoke({"conversation": conversation})
        
        # 최근 메시지만 유지
        self.messages = self.messages[-4:]
    
    def get_context(self):
        context = ""
        if self.summary:
            context = f"대화 요약: {self.summary}\n\n"
        context += "최근 대화:\n"
        for msg in self.messages:
            role = "사용자" if isinstance(msg, HumanMessage) else "AI"
            context += f"{role}: {msg.content}\n"
        return context

# 요약 메모리 테스트
llm = ChatOpenAI(model="gpt-4o-mini")
summary_memory = SummarizationMemory(llm, max_messages=6)

# 긴 대화 시뮬레이션
conversations = [
    ("안녕하세요! 제 이름은 김철수입니다.", "안녕하세요 김철수님! 만나서 반갑습니다."),
    ("저는 개발자이고 파이썬을 주로 사용합니다.", "파이썬 개발자시군요! 어떤 프로젝트를 하시나요?"),
    ("웹 개발과 데이터 분석을 합니다.", "웹 개발과 데이터 분석 모두 파이썬의 강점이죠."),
    ("최근에 머신러닝도 공부하고 있어요.", "머신러닝은 흥미로운 분야입니다. 어떤 것부터 시작하셨나요?"),
    ("scikit-learn으로 시작했습니다.", "좋은 선택입니다. scikit-learn은 입문하기 좋죠."),
    ("제가 뭘 한다고 했죠?", "웹 개발, 데이터 분석, 그리고 최근에 머신러닝을 공부하신다고 하셨습니다.")
]

for i, (human, ai) in enumerate(conversations, 1):
    print(f"\n=== 대화 {i} ===")
    print(f"사용자: {human}")
    print(f"AI: {ai}")
    summary_memory.add_exchange(human, ai)
    
    if i % 3 == 0:  # 3번째 대화마다 컨텍스트 확인
        print("\n--- 현재 메모리 상태 ---")
        print(summary_memory.get_context())


=== 대화 1 ===
사용자: 안녕하세요! 제 이름은 김철수입니다.
AI: 안녕하세요 김철수님! 만나서 반갑습니다.

=== 대화 2 ===
사용자: 저는 개발자이고 파이썬을 주로 사용합니다.
AI: 파이썬 개발자시군요! 어떤 프로젝트를 하시나요?

=== 대화 3 ===
사용자: 웹 개발과 데이터 분석을 합니다.
AI: 웹 개발과 데이터 분석 모두 파이썬의 강점이죠.

--- 현재 메모리 상태 ---
최근 대화:
사용자: 안녕하세요! 제 이름은 김철수입니다.
AI: 안녕하세요 김철수님! 만나서 반갑습니다.
사용자: 저는 개발자이고 파이썬을 주로 사용합니다.
AI: 파이썬 개발자시군요! 어떤 프로젝트를 하시나요?
사용자: 웹 개발과 데이터 분석을 합니다.
AI: 웹 개발과 데이터 분석 모두 파이썬의 강점이죠.


=== 대화 4 ===
사용자: 최근에 머신러닝도 공부하고 있어요.
AI: 머신러닝은 흥미로운 분야입니다. 어떤 것부터 시작하셨나요?

=== 대화 5 ===
사용자: scikit-learn으로 시작했습니다.
AI: 좋은 선택입니다. scikit-learn은 입문하기 좋죠.

=== 대화 6 ===
사용자: 제가 뭘 한다고 했죠?
AI: 웹 개발, 데이터 분석, 그리고 최근에 머신러닝을 공부하신다고 하셨습니다.

--- 현재 메모리 상태 ---
대화 요약: 사용자는 웹 개발과 데이터 분석을 하며 최근 머신러닝을 공부하고 있다고 말했다. AI는 파이썬의 강점을 언급하며, 머신러닝에 대한 흥미를 표현하고 사용자에게 어떤 주제로 시작했는지 질문했다.

최근 대화:
사용자: scikit-learn으로 시작했습니다.
AI: 좋은 선택입니다. scikit-learn은 입문하기 좋죠.
사용자: 제가 뭘 한다고 했죠?
AI: 웹 개발, 데이터 분석, 그리고 최근에 머신러닝을 공부하신다고 하셨습니다.



## 8. 벡터 기반 메모리

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
import datetime

class VectorMemory:
    def __init__(self, embeddings):
        self.embeddings = embeddings
        self.vectorstore = None
        self.conversation_history = []
    
    def add_conversation(self, human_msg: str, ai_msg: str):
        # 타임스탬프 추가
        timestamp = datetime.datetime.now().isoformat()
        
        # 대화를 문서로 변환
        doc = Document(
            page_content=f"사용자: {human_msg}\nAI: {ai_msg}",
            metadata={
                "timestamp": timestamp,
                "human_msg": human_msg,
                "ai_msg": ai_msg
            }
        )
        
        # 벡터 저장소에 추가
        if self.vectorstore is None:
            self.vectorstore = FAISS.from_documents([doc], self.embeddings)
        else:
            self.vectorstore.add_documents([doc])
        
        self.conversation_history.append(doc)
    
    def search_relevant_context(self, query: str, k: int = 3):
        """쿼리와 관련된 대화 검색"""
        if self.vectorstore is None:
            return []
        
        results = self.vectorstore.similarity_search(query, k=k)
        return results
    
    def get_recent_context(self, n: int = 3):
        """최근 N개 대화 반환"""
        return self.conversation_history[-n:] if self.conversation_history else []

# 벡터 메모리 테스트
embeddings = OpenAIEmbeddings()
vector_memory = VectorMemory(embeddings)

# 다양한 주제의 대화 추가
conversations = [
    ("파이썬으로 웹 개발하는 방법 알려주세요.", "Django나 Flask를 사용하면 됩니다."),
    ("머신러닝 입문 추천 도서는?", "'핸즈온 머신러닝'을 추천합니다."),
    ("좋아하는 음식은 피자입니다.", "피자는 맛있죠! 어떤 토핑을 좋아하시나요?"),
    ("데이터베이스 설계 원칙은?", "정규화, 인덱싱, 트랜잭션 관리가 중요합니다."),
    ("운동은 매일 아침에 합니다.", "아침 운동은 건강에 좋습니다!"),
]

# 대화 추가
for human, ai in conversations:
    vector_memory.add_conversation(human, ai)
    print(f"추가됨: {human[:30]}...")

# 관련 컨텍스트 검색
print("\n=== 관련 컨텍스트 검색 ===")
queries = [
    "웹 프레임워크",
    "건강한 생활",
    "데이터 관리"
]

for query in queries:
    print(f"\n쿼리: '{query}'")
    relevant = vector_memory.search_relevant_context(query, k=2)
    for i, doc in enumerate(relevant, 1):
        print(f"{i}. {doc.page_content[:100]}...")

추가됨: 파이썬으로 웹 개발하는 방법 알려주세요....
추가됨: 머신러닝 입문 추천 도서는?...
추가됨: 좋아하는 음식은 피자입니다....
추가됨: 데이터베이스 설계 원칙은?...
추가됨: 운동은 매일 아침에 합니다....

=== 관련 컨텍스트 검색 ===

쿼리: '웹 프레임워크'
1. 사용자: 파이썬으로 웹 개발하는 방법 알려주세요.
AI: Django나 Flask를 사용하면 됩니다....
2. 사용자: 데이터베이스 설계 원칙은?
AI: 정규화, 인덱싱, 트랜잭션 관리가 중요합니다....

쿼리: '건강한 생활'
1. 사용자: 운동은 매일 아침에 합니다.
AI: 아침 운동은 건강에 좋습니다!...
2. 사용자: 좋아하는 음식은 피자입니다.
AI: 피자는 맛있죠! 어떤 토핑을 좋아하시나요?...

쿼리: '데이터 관리'
1. 사용자: 데이터베이스 설계 원칙은?
AI: 정규화, 인덱싱, 트랜잭션 관리가 중요합니다....
2. 사용자: 파이썬으로 웹 개발하는 방법 알려주세요.
AI: Django나 Flask를 사용하면 됩니다....


## 실습 과제

1. 감정 상태를 추적하는 메모리 시스템 구현
2. 다중 사용자를 위한 세션 기반 메모리 관리
3. 중요도 기반 메모리 우선순위 시스템 구현

In [5]:
# 여기에 실습 코드를 작성하세요
